In [ ]:
import numpy as np
import pandas as pd
from keras.src.applications import VGG16
from keras.src.callbacks import ModelCheckpoint
from keras.src.layers import Flatten, Dense
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from tensorflow.keras.preprocessing import image
import plotly.graph_objects as go
import plotly.express as px
from keras import Sequential
from keras.src.preprocessing.image import ImageDataGenerator

In [ ]:
paths = ["./data/train", "./data/validation", "./data/test"]

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255.)

train_generator = datagen.flow_from_directory(paths[0],
                                              target_size=(150, 150),
                                              batch_size=64,
                                              class_mode="categorical")

validation_generator = datagen.flow_from_directory(paths[1],
                                                   target_size=(150, 150),
                                                   batch_size=64,
                                                   class_mode="categorical")

test_generator = datagen.flow_from_directory(paths[2],
                                             target_size=(150, 150),
                                             batch_size=1,
                                             class_mode="categorical",
                                             shuffle=False)

In [ ]:
def display_images_from_class(id):
    fnames = [f"{paths[0]}/{id}/{names}" for names in os.listdir(paths[0] + f"/{id}")]

    print(len(fnames))

    rows = (len(fnames) // 5) + 1

    plt.figure(figsize=(25, 25))
    i = 1

    for fname in fnames:
        img = image.load_img(fname, target_size=(150, 150))
        print(f"{fname}")

        plt.subplot(rows, 5, i)
        plt.imshow(img)

        i += 1


display_images_from_class(140)

In [ ]:
model = Sequential()

transfer = VGG16(input_shape=(150, 150, 3), weights='imagenet', include_top=False)
transfer.trainable = False

model.add(transfer)
model.add(Flatten())
model.add(Dense(units=len(train_generator.class_indices), activation='softmax'))

model.summary()

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

mc = ModelCheckpoint("min_loss_model.keras", monitor="val_loss", save_best_only=True, mode="min", verbose=1)
history = model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[mc])

In [ ]:
def draw_history(r):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history['accuracy'], name="accuracy"))
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history['val_accuracy'], name="val_accuracy"))
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history['loss'], name="loss"))
    fig.add_trace(go.Scatter(x=r.epoch, y=r.history['val_loss'], name="val_loss"))
    fig.show()

# draw_history(history)

In [ ]:
model.load_weights("min_loss_model.keras")
preds = model.predict(test_generator)

results = pd.DataFrame({
    "file": test_generator.filepaths,
    "true": test_generator.classes,
    "pred": np.argmax(preds, axis=1)
})

results

In [ ]:
print(f"Accuracy: {accuracy_score(results['true'], results['pred'])}")

cm = confusion_matrix(results['true'], results['pred'])
px.imshow(cm, text_auto=True)

In [ ]:
my_generator = datagen.flow_from_directory("./data/my",
                                           target_size=(150, 150),
                                           batch_size=1,
                                           shuffle=False,
                                           class_mode="categorical")

preds = model.predict(my_generator)

results = pd.DataFrame({
    "file": my_generator.filenames,
    "pred": np.argmax(preds, axis=1)
})

print(results)